In [1]:

import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

BASE = "https://www.scb.se"
URL = "https://www.scb.se/dokumentation/kvalitet-och-framtagning/"


In [2]:

html = requests.get(URL, timeout=60).text
soup = BeautifulSoup(html, "html.parser")


In [4]:

links = []
for a in soup.select("a[href]"):
    href = a["href"]
    if ".pdf" in href.lower():
        full = urljoin(BASE, href)
        text = a.get_text(" ", strip=True)
        links.append((text, full))

df = pd.DataFrame(links, columns=["linktext", "url"])


In [5]:
df

,linktext,url
0,2024 (pdf),https://www.scb.se/contentassets/a2ad4e9c34774...
1,2021 (pdf),https://www.scb.se/contentassets/a2ad4e9c34774...
2,2019 (pdf),https://www.scb.se/contentassets/a2ad4e9c34774...
3,2017 (pdf),https://www.scb.se/contentassets/a2ad4e9c34774...
4,2015 (pdf),https://www.scb.se/contentassets/a2ad4e9c34774...
...,...,...
1009,2007 (pdf),https://www.scb.se/contentassets/499655a8ec5a4...
1010,2007 (pdf),https://www.scb.se/contentassets/6e6d2915b9534...
1011,2008 (pdf),https://www.scb.se/contentassets/6e6d2915b9534...
1012,2009 (pdf),https://www.scb.se/contentassets/6e6d2915b9534...


In [6]:

# klassificera KD / StaF via url
df["typ"] = df["url"].str.lower().apply(lambda u:
    "KD" if "_kd_" in u else ("StaF" if "_staf_" in u or "framstallning" in u else "Övrigt")
)


In [8]:
df.to_csv("länkar.csv", index=False, encoding="utf-8")

In [9]:
df["fil"] = df["url"].str.split("/").str[-1]
df["produktkod"] = df["fil"].str.extract(r"^([a-z]{2}\d{4})", expand=False)



In [10]:
df.to_csv("länkar.csv", index=False, encoding="utf-8")

In [13]:

def extract_ref_year(filename: str):
    if pd.isna(filename):
        return pd.NA
    years = re.findall(r'(?:19|20)\d{2}', str(filename))
    return int(years[0]) if years else pd.NA

df["år"] = df["fil"].apply(extract_ref_year)


In [14]:
df.to_csv("länkar.csv", index=False, encoding="utf-8")